In [7]:
# connection
import os
from dotenv import load_dotenv
from shapely import Polygon, Point
import glob
import psycopg2
import pandas as pd
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import joblib

True

In [13]:
df = pd.read_parquet("dataset_vectorize.parquet")

In [ ]:
# Step 1: Load Data
# df = pd.read_csv('poi_data.csv')  # Replace with your actual dataset file
X = df['poi_name']  # Input: POI name
y = df[['brand_name','category_name', 'group_name', 'industry_name']]  # Targets

# Step 2: Data Splitting
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 3: Text Vectorization using TF-IDF
vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1, 2), stop_words='english')
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Step 4: Define Model and Hyperparameter Tuning
base_model = RandomForestClassifier(random_state=42)

# Hyperparameter grid
param_grid = {
    'estimator__n_estimators': [50, 100, 150],
    'estimator__max_depth': [None, 10, 20],
    'estimator__min_samples_split': [2, 5, 10]
}

multi_output_model = MultiOutputClassifier(base_model)

# Use GridSearchCV for hyperparameter tuning
grid_search = GridSearchCV(
    estimator=multi_output_model,
    param_grid=param_grid,
    scoring='accuracy',  # You can use other metrics such as f1_macro
    cv=3,  # 3-fold cross-validation
    verbose=1,
    n_jobs=-1
)

# Train the model with hyperparameter tuning
print("Starting model tuning...")
grid_search.fit(X_train_tfidf, y_train)

# Best model after tuning
best_model = grid_search.best_estimator_
print(f"Best Parameters: {grid_search.best_params_}")

# Step 5: Model Testing
y_pred = best_model.predict(X_test_tfidf)

# Evaluate the model for each target
for i, target in enumerate(['category_name', 'industry_name', 'group_name']):
    print(f"\nClassification Report for {target}:")
    print(classification_report(y_test.iloc[:, i], y_pred[:, i]))

# Step 6: Save the Best Model and TF-IDF Vectorizer
joblib.dump(best_model, 'poi_best_multioutput_model.pkl')
joblib.dump(vectorizer, 'poi_tfidf_vectorizer.pkl')
print("Model and vectorizer saved successfully!")

